In [1]:
import csv
import json
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor,as_completed

In [2]:
base_page_url = "https://emlak.az/elanlar/?ann_type=1&tip[]=1&sort_type=0&page="

In [49]:
def scrapePageUrl(base_page_url,page):
    url = base_page_url+str(page)
    print("Getting :", url)
    links = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text,"html.parser")
        ticket_list = soup.select_one(".ticket-list").select(".ticket")
        for ticket in ticket_list:
            a = ticket.find("a")
            links.append(a['href'])
    except BaseException as error:
        print(error)
    
    return links

In [63]:
# all_links = []

# for page in range(1,239):
#     links = scrapePageUrl(base_page_url,page)
#     all_links.extend(links)
    
# print(len(all_links))

In [56]:
print(all_links[1])
print(all_links[-1])

/861448-satilir-2-otaqli-108-m2-yeni-tikili-caspian-plaza.html
/857941-satilir-3-otaqli-110-m2-yeni-tikili-ehmedli.html


In [62]:
# with open("yeni_tikili_857941_861448.txt","w") as f:
#     for link in all_links:
#         f.write(f"{link}\n")

In [101]:
base_url = "https://emlak.az/"
# url = "868080"
url = "860001"

In [102]:
def parseParams(parameters):
    params = {}
    for param in parameters:
        if len(param) == 1:
            continue
        res = re.search(r"<dd><span class=\"label\">(.+)<\/span>(.+)<\/dd>",str(param))
        params[res.group(1)] = res.group(2)
        
    return params


def scrapeProperty(base_url,uri):
    item = {
        "item_id": int(uri.split("-")[0])
    }
    try:
        url = base_url + uri + "-.html"
        response = requests.get(url)
        soup = BeautifulSoup(response.text,"html.parser")

        title = soup.select_one(".panel > h1.title").getText()
        item["title"] = title
        item["isRent"] = title[0] != "S"
        item["date"] = soup.select_one(".box-panel > .date > strong").getText()
        item["price"] = int("".join(soup.select_one(".left-bar > .price > .m").getText().split(" ")))
        item["address"] = soup.select_one(".map-address > h4").getText().split(": ")[1]
        item["phone"] = soup.select_one(".silver-box > .phone").getText()
        
        # coords (lat,lng)
        coords = soup.select_one("#google_map")["value"].strip("()").split(", ")
        item["lat"] = float(coords[0]) 
        item["lng"] = float(coords[1]) 
        
        # params
        params = parseParams(soup.select_one(".technical-characteristics"))
        item["type"] = params["Əmlakın növü"]
        item["area"] = int(params["Sahə"].split(" ")[0])
        item["rooms"] = int(params["Otaqların sayı"])
        item["floor"] = int(params["Yerləşdiyi mərtəbə"])
        item["floors"] = int(params["Mərtəbə sayı"])
        item["renovation"] = params["Təmiri"]
        item["document"] = params["Sənədin tipi"]
        
        item["description"] = soup.select_one(".desc > p").getText()
        return item
    except BaseException as error:
        print(error)

item = scrapeProperty(base_url,url)
# print(json.dumps(item,indent=2,ensure_ascii=False))
item

'Yerləşdiyi mərtəbə'


In [103]:
obj= {
    "a" :1,
    "b": 2
}

In [109]:
obj.keys()

dict_keys(['a', 'b'])

In [111]:
print("a" if "c" in obj.keys() else "error")

error
